### Text Selection Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Turkish
Lang Id = 2
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Result/3-Text Selection Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Text Selection/Result/2-Text Selection Data Merge"

In [9]:
#df_word = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx")
df_word = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Text_Selection_Result.csv")
df_word = df_word.fillna("[]")
df_word = df_word.iloc[990:1020,]
df_word

,word,frequency,twogram,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
990,herşeyi,71175,"['için herşeyi', 'bu herşeyi', 'bana herşeyi',...","['için herşeyi yaparım', 'senin için herşeyi',...","['senin için herşeyi yaparım', 'için elimden g...","['böylece hızlandırmak için herşeyi yaparsın',...","['adam zihnini meşgul edebilmek için herşeyi',...",['bir adam zihnini meşgul edebilmek için herşe...,['bir adam zihnini meşgul edebilmek için herşe...,['hapishanede bir adam zihnini meşgul edebilme...,['herşeyi düzeltmem için bana bel bağladı ve b...
991,görev,71055,"['bir görev', 'görev tamamlandı', 'bu görev', ...","['bir görev için', 'bu görev için', 'önemli bi...","['çok önemli bir görev', 'sana bir görev verec...","['biz tanrı adına görev yapıyoruz', 'evet size...","['bana çok önemli bir görev verdi', 'dönmeden ...",['dönmeden önce yapmanı istediğim bir görev da...,['dönmeden önce yapmanı istediğim bir görev da...,['evine dönmeden önce yapmanı istediğim bir gö...,['efendim görev yerimi nöbeti teslim ettikten ...
992,getirdim,70790,"['şey getirdim', 'hediye getirdim', 'buraya ge...","['bir şey getirdim', 'bir hediye getirdim', 'b...","['sana bir şey getirdim', 'sana bir hediye get...","['senin için bir şey getirdim', 'sana yiyecek ...","['bu yüzden çocukları da yanımda getirdim', 'ç...",['şimdi onun buraya dönmesine izin mi vereceks...,['onu tedavi etmesi için bir aylığına buraya g...,['bunu mümkün kılmak için düzenli bir alışkanl...,['ve korkunç bir şekilde intikam almak için on...
993,dinleyin,70632,"['beni dinleyin', 'dinleyin beni', 'şunu dinle...","['beni iyi dinleyin', 'lütfen beni dinleyin', ...","['şimdi beni iyi dinleyin', 'bir de şunu dinle...","['dinleyin orospu çocukları adi herifler', 'sa...","['dinleyin ben de bir tane alacağım', 'dinleyi...","['dinleyin bir tek değişmez var tek bir', 'din...",['dinleyin bir tek değişmez var tek bir evrens...,['dinleyin sizin için bu akşam olağanüstü bir ...,['benim bir fikrim var dinleyin bu seferki çok...
994,hale,70625,"['nasıl öldüğünü', 'bir hız', 'tam hız', 'şu h...","['şu hale bak', 'nasıl bu hale', 'hale getirme...","['etkisiz hale getirmek için', 'nasıl bu hale ...","['ne hale geldiğine bir bak', 'seni bu hale o ...","['ne zaman savaşmaya hazır hale gelebilirler',...",['erken ne zaman savaşmaya hazır hale gelebili...,['en erken ne zaman savaşmaya hazır hale geleb...,['beraberce bu şehri saygın halkımız için güve...,['gecenin sesleri öyle tanıdık hale geldi ki h...
995,kavga,70611,"['kavga mı', 'kavga etmek', 'kavga ettik', 'bi...","['kavga etmek istemiyorum', 'kavga mı ettiniz'...","['seninle kavga etmek istemiyorum', 'kavga etm...","['benimle kavga etmek mi istiyorsun', 'buraya ...","['bu sadece başlangıç kavga devam ediyor', 'ka...",['kara panter partisinin ortasında kavga ettiğ...,['kara panter partisinin ortasında kavga ettiğ...,['neden mi bilmiyorum daha önce hiç kavga etme...,['neden mi bilmiyorum daha önce hiç kavga etme...
996,arabayı,70310,"['arabayı durdur', 'bu arabayı', 'durdur araba...","['durdur şu arabayı', 'ben arabayı getireyim',...","['şu arabayı takip et', 'arabayı park ettim ve...","['arabayı park ettim ve bekledim', 'o da bu ar...",['akşam arabayı bırakırken arka koltuğu temizl...,['her akşam arabayı bırakırken arka koltuğu te...,"['bildiğimiz bir şey varsa o da bu arabayı', '...",['bildiğimiz bir şey varsa o da bu arabayı ist...,['bu sadece bir ya da iki kötü adamın işi deği...
997,oldun,70121,"['oldun mu', 'mı oldun', 'yardımcı oldun', 'mu...","['onlar da kendi', 'mutlu oldun mu', 'tatmin o...","['hiç aşık oldun mu', 'buna sen sebep oldun', ...","['sen artık yeni biri oldun', 'sen hiç aşık ol...","['hiç bir kadınla birlikte oldun mu', 'hayatın...","['hiç bir kızla birlikte oldun mu forrest', 'k...",['bana karşı candan ve sadık bir arkadaş oldun...,['şu anda canlı yayındayız ve sen bir numara o...,['buradan ayrılmak üzere planımız vardı ama se...
998,bölüm,69859,"['gelecek bölüm', 'bir bölüm', 'sonraki bölüm'...","['bir sonraki

In [10]:
df_word.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 990 to 1019
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   word       30 non-null     object
 1   frequency  30 non-null     int64 
 2   twogram    30 non-null     object
 3   threegram  30 non-null     object
 4   fourgram   30 non-null     object
 5   fivegram   30 non-null     object
 6   sixgram    30 non-null     object
 7   sevengram  30 non-null     object
 8   eightgram  30 non-null     object
 9   ninegram   30 non-null     object
 10  tengram    30 non-null     object
dtypes: int64(1), object(10)
memory usage: 2.7+ KB


In [11]:
#df_twogram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx")
df_twogram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Twogram_Text_Selection_Result.csv")
df_twogram = df_twogram.fillna("[]")
df_twogram = df_twogram.iloc[0:100,]
df_twogram

,twogram,frequency,threegram,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey,859944,"['bir şey yok', 'bir şey var', 'başka bir şey'...","['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'sana bir şey sorabil...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ve eğer o arkadaşı bulursa onu asla bırakmay...,['bana sadece içinde şeker olmayan bir şey ver...
1,değil mi,585879,"['öyle değil mi', 'biliyorsun değil mi', 'güze...","['de öyle değil mi', 'bir şey değil mi', 'çok ...","['sence de öyle değil mi', 'sizce de öyle deği...","['bu iyi bir şey değil mi', 'ne demek istediği...","['o kadar da kötü değildi değil mi', 'o kadar ...",['bir kadının şarkı söylemesi oldukça şaşırtıc...,['evimde bir kadının şarkı söylemesi oldukça ş...,['ama bugün yeni bir maç var galiba değil mi m...
2,ben de,377765,"['ben de seni', 'ben de öyle', 've ben de', 'e...","['ben de seni seviyorum', 'ben de öyle düşünmü...","['ben de seni gördüğüme sevindim', 'evet ben d...","['ben de en az senin kadar', 'ben de sana aynı...","['ben de sizi karı koca ilan ediyorum', 'ben d...",['siobhan intihar etti ben de onun yerine geçt...,['siz formları alırsınız ben de onları hazırla...,['ne dediğimi duymadın mı evlat ben de ona eve...
3,teşekkür ederim,370619,"['çok teşekkür ederim', 'için teşekkür ederim'...","['için çok teşekkür ederim', 'geldiğiniz için ...","['her şey için teşekkür ederim', 'zaman ayırdı...","['her şey için çok teşekkür ederim', 'beni kab...",['olmama izin verdiğin için tekrar teşekkür ed...,['gibi olmama izin verdiğin için tekrar teşekk...,['kendim gibi olmama izin verdiğin için tekrar...,['önce parfüm mağazasından alınmış video kaydı...
4,ne oldu,322758,"['sonra ne oldu', 'ne oldu sana', 'sana ne old...","['bil bakalım ne oldu', 'sana ne oldu böyle', ...","['az önce ne oldu öyle', 'sonra ne oldu biliyo...","['ne oldu bir sorun mu var', 'az önce ne oldu ...",['gelecek hakkındaki bilgileri reddetme konuşm...,['insanlık nereye gidiyoruz ve niçin soruların...,['ya insanlık nereye gidiyoruz ve niçin sorula...,['burnuna ne oldu gittes yoksa kapı yerine bac...
...,...,...,...,...,...,...,...,...,...,...
95,hadi gidelim,61646,"['hadi gidelim buradan', 'tamam hadi gidelim',...","['hadi gidelim o zaman', 'hadi gidelim hadi gi...","['hayalimdeki iş buydu hadi gidelim', 'hadi gi...","['hadi gidelim korkunç şeylerin olduğu yere', ...","['siz de ateşi üzerinize çekin hadi gidelim', ...",['arada siz de ateşi üzerinize çekin hadi gide...,['bu arada siz de ateşi üzerinize çekin hadi g...,['hadi gidelim bebeğim bebeğim oh biz tek bir ...
96,ama bu,61607,"['evet ama bu', 'ama bu sefer', 'ama bu çok', ...","['ama bu doğru değil', 'ama bu sadece bir', 'ö...","['ama bu uzun zaman önceydi', 'ama bu iyi bir ...","['bir sürü tüfek var ama bu', 'ama bu çok uzun...","['ama bu iyi bir şey değil mi', 'bir sürü tüfe...","['bir sürü tüfek var ama bu tüfek benim', 'söy...",['bunu söyleyip duruyorlar ama bu bizim için ö...,['sana yardım etmek isterim ama bu yüzden karı...
97,bir gün,61602,"['güzel bir gün', 'bir gün daha', 'belki bir g...","['bir gün değil mi', 'çok güzel bir gün', 'ne ...","['güzel bir gün değil mi', 'ne kadar güzel bir...","['ne güzel bir gün değil mi', 'neşeli ruhu bir...","['neşeli ruhu bir gün yelken açtı ta', 'ruhu b...",['neşeli ruhu bir gün yelken açtı ta denizden'...,['bir gün yalan açığa çıkacak ve karanlığın iç...,['bir gün yalan açığa çıkacak ve karanlığın iç...
98,iyi misin,61541,"['sen iyi misin', 'daha iyi misin', 'hey iyi m...","['hey sen iyi misin', 'şimdi daha iyi misin', ...","['hey hey sen iyi misin', 'gel buraya bebeğim ...","['ah iyi misin ahbap yapma bana', 'yerinde diy...","['de yerinde diye benden iyi misin yani', 'sab...",['bu sabah gerçekten tuhaf davranıyorsun sen i...,['kart ya da o tür oyunlarda gerçekten iyi mis...,[]


In [12]:
#df_threegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx")
df_threegram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Threegram_Text_Selection_Result.csv")
df_threegram = df_threegram.fillna("[]")
df_threegram = df_threegram.iloc[0:100,]
df_threegram

,threegram,frequency,fourgram,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey yok,113165,"['diye bir şey yok', 'başka bir şey yok', 'bir...","['biz diye bir şey yok', 'başka bir şey yok mu...","['bir şey yok bir şey yok', 'yok bir şey yok b...","['söylemek istediğin başka bir şey yok mu', 'b...",['birbirimiz olmadan ikimizin de yapacağı bir ...,['ölmeden önce de aşıktım ve yapabileceğim bir...,['o ölmeden önce de aşıktım ve yapabileceğim b...
1,bir şey var,110455,"['bir şey var mı', 'gereken bir şey var', 'baş...","['başka bir şey var mı', 'yapabileceğim bir şe...","['sana söylemem gereken bir şey var', 'söyleme...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['ev sahibi babamız için bende özel bir şey va...,['kaosun içinde düzen için mücadele eden tek b...
2,bu da ne,89463,"['bu da ne demek', 'bu da ne böyle', 'bu da ne...","['bu da ne demek oluyor', 'bu da ne demek şimd...","['bu da ne demek oluyor şimdi', 'bu da ne deme...","['bu da ne biçim bir soru böyle', 'ama bu da n...","['alınmayın ama bu da ne demek oluyor şimdi', ...",[],['bu da ne olursa olsun oynamaya karar verdiği...
3,her şey yolunda,75968,"['her şey yolunda mı', 'her şey yolunda gidece...","['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda her şey yolunda', 'merak etm...",['helikopterler bizi koruyacak her şey yolunda...,"['her şey yolunda giderse çok yakında bir ev',...",['her şey yolunda giderse çok yakında bir ev a...,['dostlar acil bir durum yaşadık ama şimdi her...
4,başka bir şey,75193,"['başka bir şey var', 'başka bir şey yok', 'ba...","['başka bir şey var mı', 'başka bir şey daha v...","['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['kılıcını ve onun kılıcını düşün başka bir şe...,['sabırsızlıkla beklediğim zevkle yapacağım ba...
...,...,...,...,...,...,...,...,...,...
95,çok büyük bir,16228,"['bu çok büyük bir', 'çok büyük bir hata', 'iç...","['çok büyük bir hata yapıyorsun', 'çok büyük b...","['bu benim için çok büyük bir', 'bir kişi için...","['bir kişi için çok büyük bir güç', 'önümüzdek...",['bu çok büyük bir saçmalık buraya bunun için'...,['çok büyük bir saçmalık buraya bunun için gel...,['bu çok büyük bir saçmalık buraya bunun için ...
96,çok daha iyi,16197,"['çok daha iyi bir', 'çok daha iyi hissediyoru...","['kendimi çok daha iyi hissediyorum', 'şimdi ç...","['şimdi kendimi çok daha iyi hissediyorum', 'd...",['oldukları yerden durumu bizden çok daha iyi'...,['oldukları yerden durumu bizden çok daha iyi ...,['ve umduğumdan da çok daha iyi bir jedi oldun...,['sanırım ben senden çok daha iyi bir özel aja...
97,peki ya sen,16151,"['peki ya sen ne', 'peki ya sen ve', 'peki ya ...","['peki ya sen ne yapıyordun', 'peki ya sen ne ...","['uğruna ölmeye hazırım peki ya sen', 'peki ya...","['ülkem uğruna ölmeye hazırım peki ya sen', 'p...",[],[],[]
98,iyi bir şey,16038,"['bu iyi bir şey', 'iyi bir şey değil', 'daha ...","['bu iyi bir şey değil', 'bu iyi bir şey mi', ...","['bu iyi bir şey değil mi', 'ama bu iyi bir şe...","['ama bu iyi bir şey değil mi', 'iyi bir şey m...","['iyi bir şey mi kötü bir şey mi', 'bu iyi bir...","['bu iyi bir şey mi kötü bir şey mi', 'çünkü e...",['sanırım ikimiz için de yalnız kalmak iyi bir...


In [13]:
#df_fourgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx")
df_fourgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.csv")
df_fourgram = df_fourgram.fillna("[]")
df_fourgram = df_fourgram.iloc[0:100,]
df_fourgram

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,fourgram,frequency,fivegram,sixgram,sevengram,eightgram,ninegram,tengram
0,bir şey var mı,41773,"['başka bir şey var mı', 'yapabileceğim bir şe...","['söylemek istediğin bir şey var mı', 'için ya...","['bana söylemek istediğin bir şey var mı', 'se...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['tanrı aşkına çalışmak için yaşamak diye bir ...
1,her şey yolunda mı,31126,"['orada her şey yolunda mı', 'burada her şey y...","['her şey yolunda mı diye bakmak', 'her şey yo...","['her şey yolunda mı diye bakmak istedim', 'he...","['git de her şey yolunda mı bir bak', 'hadi gi...",['hadi git de her şey yolunda mı bir bak'],['yukarı gelip her şey yolunda mı diye kontrol...
2,burada ne işin var,21993,"['senin burada ne işin var', 'burada ne işin v...","['o zaman burada ne işin var', 'bu saatte bura...","['gecenin bu saatinde burada ne işin var', 'on...",['o kadar paran varsa burada ne işin var'],[],[]
3,bir sorun mu var,21423,"['bir sorun mu var efendim', 'ilgili bir sorun...","['bir sorun mu var memur bey', 'ile ilgili bir...","['yerçekimi ile ilgili bir sorun mu var', 'dem...",['küresel yerçekimi ile ilgili bir sorun mu va...,['neden demin geldiğimde bir sorun mu var diye...,['merak ediyorum da acaba bende bir sorun mu v...
4,ben de seni seviyorum,17338,"['ben de seni seviyorum baba', 'ben de seni se...","['ben de seni seviyorum biliyorsun değil', 'be...","['ben de seni seviyorum biliyorsun değil mi', ...","['bu çok güzel çünkü ben de seni seviyorum', '...",[],['ben de seni seviyorum tatlım hem de her şeyd...
...,...,...,...,...,...,...,...,...
95,bana bir iyilik yapar,3367,"['bana bir iyilik yapar mısın', 'bana bir iyil...","['dinle bana bir iyilik yapar mısın', 'bana bi...","['yok mu bana bir iyilik yapar mısın', 'o zama...",['aklıma geldi de bana bir iyilik yapar mısın'],[],[]
96,için her şeyi yaparım,3342,"['senin için her şeyi yaparım', 'onun için her...","['geri almak için her şeyi yaparım', 'yardım e...","['onu geri almak için her şeyi yaparım', 'onu ...",['kalmak için her şeyi yaparım ama gitmek zoru...,['burada kalmak için her şeyi yaparım ama gitm...,['gittiğimi biliyorum ama bunu telafi etmek iç...
97,bir şey söyleyebilir miyim,3326,"['sana bir şey söyleyebilir miyim', 'size bir ...","['sana kişisel bir şey söyleyebilir miyim', 'ö...",['önce kimseye söylemediğim bir şey söyleyebil...,['daha önce kimseye söylemediğim bir şey söyle...,['sana daha önce kimseye söylemediğim bir şey ...,[]
98,rahatsız ettiğim için üzgünüm,3320,"['rahatsız ettiğim için üzgünüm ama', 'sizi ra...","['bu saatte rahatsız ettiğim için üzgünüm', 'g...",['bayanlar ve baylar rahatsız ettiğim için üzg...,['geç saatte rahatsız ettiğim için üzgünüm ama...,['geç saatte rahatsız ettiğim için üzgünüm ama...,['rahatsız ettiğim için üzgünüm ikimizin de ya...


In [14]:
#df_fivegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx")
df_fivegram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.csv")
df_fivegram = df_fivegram.fillna("[]")
df_fivegram = df_fivegram.iloc[0:100,]
df_fivegram

,fivegram,frequency,sixgram,sevengram,eightgram,ninegram,tengram
0,başka bir şey var mı,14104,"['istediğin başka bir şey var mı', 'yapabilece...","['için yapabileceğim başka bir şey var mı', 'b...",['sizin için yapabileceğim başka bir şey var m...,['sizin için yapabileceğim başka bir şey var m...,['affedersiniz bayan sizin için yapabileceğim ...
1,bu da ne demek oluyor,10205,"['bu da ne demek oluyor şimdi', 'bu da ne deme...","['ama bu da ne demek oluyor şimdi', 'alınmayın...",['alınmayın ama bu da ne demek oluyor şimdi'],[],['bu da ne demek oluyor yargılamak gibi bir şe...
2,o kadar da kötü değil,7012,"['bu o kadar da kötü değil', 'o kadar da kötü ...","['o kadar da kötü değil değil mi', 'hadi ama o...","['bu o kadar da kötü değil değil mi', 'ama o k...",['hadi ama pascal burası o kadar da kötü değil...,['hem prenses olmak o kadar da kötü değil deği...
3,sence de öyle değil mi,6305,"['görünüyor sence de öyle değil mi', 'var senc...","['iyi gidiyor sence de öyle değil mi', 'zamand...","['hoş bir vadi sence de öyle değil mi', 'çok d...","['burası hoş bir vadi sence de öyle değil mi',...",['gelecek geçmişten çok daha ilginç sence de ö...
4,sana bir şey sorabilir miyim,6224,"['hey sana bir şey sorabilir miyim', 'baba san...",['kimseye söylemezsen sana bir şey sorabilir m...,[],[],[]
...,...,...,...,...,...,...,...
95,biz diye bir şey yok,1356,"['artık biz diye bir şey yok', 'biz diye bir ş...","['biz diye bir şey yok tamam mı', 'bu sefer bi...","['bu sefer biz diye bir şey yok dostum', 'defa...",['son defa söylüyorum diğer biz diye bir şey y...,[]
96,için yapabileceğim bir şey var,1346,"['için yapabileceğim bir şey var mı', 'senin i...","['senin için yapabileceğim bir şey var mı', 's...",['yardım etmek için yapabileceğim bir şey var ...,['bunu telafi etmek için yapabileceğim bir şey...,['bayan woodhouse babanız için yapabileceğim b...
97,bu iyi bir fikir mi,1324,"['sence bu iyi bir fikir mi', 'bu iyi bir fiki...",[],[],[],[]
98,göstermek istediğim bir şey var,1311,"['sana göstermek istediğim bir şey var', 'size...","['gel sana göstermek istediğim bir şey var', '...",['dışarıda size göstermek istediğim bir şey va...,['size dışarda göstermek istediğim bir şey var...,[]


In [15]:
#df_sixgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx")
df_sixgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.csv")
df_sixgram = df_sixgram.fillna("[]")
df_sixgram = df_sixgram.iloc[0:100,]
df_sixgram

,sixgram,frequency,sevengram,eightgram,ninegram,tengram
0,sana söylemem gereken bir şey var,3646,"['ama sana söylemem gereken bir şey var', 'san...","['benim de sana söylemem gereken bir şey var',...",['sana söylemem gereken bir şey var ama nasıl ...,['sana söylemem gereken bir şey var ama nasıl ...
1,bir iki üç dört beş altı,2025,"['bir iki üç dört beş altı yedi', 've bir iki ...","['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,söylemek istediğin bir şey var mı,1794,"['bana söylemek istediğin bir şey var mı', 'bi...",['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,daha iyi bir fikrin var mı,1792,"['senin daha iyi bir fikrin var mı', 'peki dah...","['peki daha iyi bir fikrin var mı şerif', 'lan...",[],['benim dahil olmadığım daha iyi bir fikrin va...
4,bunun iyi bir fikir olduğunu sanmıyorum,1743,['ama bunun iyi bir fikir olduğunu sanmıyorum'...,['hayır hayır bunun iyi bir fikir olduğunu san...,[],[]
...,...,...,...,...,...,...
95,size göstermek istediğim bir şey var,270,['dışarıda size göstermek istediğim bir şey va...,['dışarıda size göstermek istediğim bir şey va...,[],[]
96,önce hiç böyle bir şey görmedim,263,['daha önce hiç böyle bir şey görmedim'],['ben daha önce hiç böyle bir şey görmedim'],[],[]
97,gökte olduğu gibi yeryüzünde de senin,261,['gökte olduğu gibi yeryüzünde de senin istedi...,['gökte olduğu gibi yeryüzünde de senin istedi...,['gelsin gökte olduğu gibi yeryüzünde de senin...,[]
98,istediğiniz başka bir şey var mı,258,"['söylemek istediğiniz başka bir şey var mı', ...",['bana söylemek istediğiniz başka bir şey var ...,['devam etmeden açıklamak istediğiniz başka bi...,[]


In [16]:
#df_sevengram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx")
df_sevengram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.csv")
df_sevengram = df_sevengram.fillna("[]")
df_sevengram = df_sevengram.iloc[0:100,]
df_sevengram

,sevengram,frequency,eightgram,ninegram,tengram
0,bir iki üç dört beş altı yedi,1455,"['bir iki üç dört beş altı yedi sekiz', 've bi...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,iki üç dört beş altı yedi sekiz,1080,"['bir iki üç dört beş altı yedi sekiz', 'iki ü...","['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
2,bana söylemek istediğin bir şey var mı,752,['önce bana söylemek istediğin bir şey var mı'...,['çıkmadan önce bana söylemek istediğin bir şe...,['arabadan çıkmadan önce bana söylemek istediğ...
3,yedi altı beş dört üç iki bir,596,"['sekiz yedi altı beş dört üç iki bir', 'yedi ...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
4,sekiz yedi altı beş dört üç iki,551,"['dokuz sekiz yedi altı beş dört üç iki', 'sek...","['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
...,...,...,...,...,...
95,hem iyi hem de kötü haberlerim var,97,"['beyler hem iyi hem de kötü haberlerim var', ...",['senin için hem iyi hem de kötü haberlerim var'],['galiba senin için hem iyi hem de kötü haberl...
96,ne olduğu hakkında bir fikriniz var mı,97,['şeyin ne olduğu hakkında bir fikriniz var mı'],['bu şeyin ne olduğu hakkında bir fikriniz var...,[]
97,müthiş bir güce sahip bir kalbim var,96,['çünkü müthiş bir güce sahip bir kalbim var'],[],[]
98,bunun senin için ne kadar önemli olduğunu,95,['bunun senin için ne kadar önemli olduğunu bi...,[],['bunun senin için ne kadar önemli olduğunu bi...


In [17]:
#df_eightgram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx")
df_eightgram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.csv")
df_eightgram = df_eightgram.fillna("[]")
df_eightgram = df_eightgram.iloc[0:100,]
df_eightgram

,eightgram,frequency,ninegram,tengram
0,bir iki üç dört beş altı yedi sekiz,1060,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
1,dokuz sekiz yedi altı beş dört üç iki,513,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
2,sekiz yedi altı beş dört üç iki bir,497,"['dokuz sekiz yedi altı beş dört üç iki bir', ...",['on dokuz sekiz yedi altı beş dört üç iki bir...
3,iki üç dört beş altı yedi sekiz dokuz,411,"['bir iki üç dört beş altı yedi sekiz dokuz', ...",['bir iki üç dört beş altı yedi sekiz dokuz on...
4,üç dört beş altı yedi sekiz dokuz on,329,"['iki üç dört beş altı yedi sekiz dokuz on', '...",['bir iki üç dört beş altı yedi sekiz dokuz on...
...,...,...,...,...
95,eğer dünya değişirse beni hiçbir şey bilmediği...,50,['eğer dünya değişirse beni hiçbir şey bilmedi...,['ama eğer dünya değişirse beni hiçbir şey bil...
96,dört ayaklı ve her zaman harekete hazır olan,50,['dört ayaklı ve her zaman harekete hazır olan...,['dört ayaklı ve her zaman harekete hazır olan...
97,ayaklı ve her zaman harekete hazır olan nedir,50,['dört ayaklı ve her zaman harekete hazır olan...,['dört ayaklı ve her zaman harekete hazır olan...
98,bu işlemi bu akşam başlatmak istiyorsak yapaca...,50,['bu işlemi bu akşam başlatmak istiyorsak yapa...,['bu işlemi bu akşam başlatmak istiyorsak yapa...


In [18]:
#df_ninegram = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx")
df_ninegram = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.csv")
df_ninegram = df_ninegram.fillna("[]")
df_ninegram = df_ninegram.iloc[0:100,]
df_ninegram

,ninegram,frequency,tengram
0,dokuz sekiz yedi altı beş dört üç iki bir,463,['on dokuz sekiz yedi altı beş dört üç iki bir...
1,bir iki üç dört beş altı yedi sekiz dokuz,405,['bir iki üç dört beş altı yedi sekiz dokuz on...
2,iki üç dört beş altı yedi sekiz dokuz on,287,['bir iki üç dört beş altı yedi sekiz dokuz on']
3,on dokuz sekiz yedi altı beş dört üç iki,277,['on dokuz sekiz yedi altı beş dört üç iki bir']
4,i t a r i s y e n,128,['g i t a r i s y e n']
...,...,...,...
95,onu onu tekrar elime almayı çok isterdim son bir,35,['onu onu tekrar elime almayı çok isterdim son...
96,her şeyi olduğu gibi bırakın ve bırakın sözler...,35,['her şeyi olduğu gibi bırakın ve bırakın sözl...
97,kazanırsak tüm kazancı sen alırsın ihtiyacım o...,35,['eğer kazanırsak tüm kazancı sen alırsın ihti...
98,onu tekrar elime almayı çok isterdim son bir kez,35,['onu onu tekrar elime almayı çok isterdim son...


#### Sending Data To Firestore

##### Photo Data

In [19]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Text Data

##### Word

In [20]:
df_word["twogram"] = df_word["twogram"].apply(eval)
df_word["threegram"] = df_word["threegram"].apply(eval)
df_word["fourgram"] = df_word["fourgram"].apply(eval)
df_word["fivegram"] = df_word["fivegram"].apply(eval)
df_word["sixgram"] = df_word["sixgram"].apply(eval)
df_word["sevengram"] = df_word["sevengram"].apply(eval)
df_word["eightgram"] = df_word["eightgram"].apply(eval)
df_word["ninegram"] = df_word["ninegram"].apply(eval)
df_word["tengram"] = df_word["tengram"].apply(eval)

In [21]:
word = df_word["word"].to_list()
frequency_list = df_word["frequency"].to_list()
twogram_list = df_word["twogram"].to_list()
threegram_list = df_word["threegram"].to_list()
fourgram_list = df_word["fourgram"].to_list()
fivegram_list = df_word["fivegram"].to_list()
sixgram_list = df_word["sixgram"].to_list()
sevengram_list = df_word["sevengram"].to_list()
eightgram_list = df_word["eightgram"].to_list()
ninegram_list = df_word["ninegram"].to_list()
tengram_list = df_word["tengram"].to_list()

In [22]:
firestore_word_data = zip(word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [63]:
#word_data_list = []
#for word,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
#    data = {f"{word}":{"ninegram":ninegram_list,"tengram":tengram_list}}
#    
#    word_data_list.append({f"{word}":data})

In [23]:
word_dict = {}
for word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
    dict_var = {f"{word}":{"photo":"","frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    word_dict.update(dict_var)     

In [45]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").set(word_dict)

InvalidArgument: 400 Document 'projects/flashcard-1c46e/databases/(default)/documents/text_selection/Turkish/word_data/details' cannot be written because its size (3,483,669 bytes) exceeds the maximum allowed size of 1,048,576 bytes.

In [ ]:
#fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").add(word_dict)

In [23]:
# Other way for document file exceed limit
non_alfa_num_list = []
for word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
    word_dict = {}
    dict_var = {f"{word}":{"photo":"","frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    #word_dict.update(dict_var)
    #fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").set(dict_var)
    try:
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(dict_var)
    except:
        non_alfa_num_list.append(word)  

In [ ]:
# Other way for document file exceed limit will test
word_dict = {}
i = 0
for word,frequency_list,twogram_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_word_data:
    dict_var = {f"{word}":{"photo":"","frequency":frequency_list,"twogram":twogram_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    word_dict.update(dict_var)
    if i == 2:
        fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(word_dict)
        word_dict = {}
        i = 0
    else:
        i+=1
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(word_dict)


##### Update

In [ ]:
update_word = "hoşuma"

In [ ]:
dict_update = {f"{update_word}.photo":"www.store4.com",f"{update_word}.other":["gitti",{"zıt":"gelmek"}]}

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("word_data").document(f"details").update(dict_update)

##### Twogram

In [ ]:
df_twogram

In [22]:
df_twogram["threegram"] = df_twogram["threegram"].apply(eval)
df_twogram["fourgram"] = df_twogram["fourgram"].apply(eval)
df_twogram["fivegram"] = df_twogram["fivegram"].apply(eval)
df_twogram["sixgram"] = df_twogram["sixgram"].apply(eval)
df_twogram["sevengram"] = df_twogram["sevengram"].apply(eval)
df_twogram["eightgram"] = df_twogram["eightgram"].apply(eval)
df_twogram["ninegram"] = df_twogram["ninegram"].apply(eval)
df_twogram["tengram"] = df_twogram["tengram"].apply(eval)

In [23]:
#twogram = df_twogram["twogram"].values.tolist()

In [24]:
twogram = df_twogram["twogram"].to_list()
frequency_list = df_twogram["frequency"].to_list()
threegram_list = df_twogram["threegram"].to_list()
fourgram_list = df_twogram["fourgram"].to_list()
fivegram_list = df_twogram["fivegram"].to_list()
sixgram_list = df_twogram["sixgram"].to_list()
sevengram_list = df_twogram["sevengram"].to_list()
eightgram_list = df_twogram["eightgram"].to_list()
ninegram_list = df_twogram["ninegram"].to_list()
tengram_list = df_twogram["tengram"].to_list()

In [ ]:
firestore_twogram_data = zip(twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
twogram_dict = {}
for twogram,frequency_list,threegram_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_twogram_data:
    dict_var = {f"{twogram}":{"photo":"","frequency":frequency_list,"threegram":threegram_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    twogram_dict.update(dict_var)    

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("twogram_data").document(f"details").set(twogram_dict)

##### Threegram

In [ ]:
df_threegram

In [25]:
df_threegram["fourgram"] = df_threegram["fourgram"].apply(eval)
df_threegram["fivegram"] = df_threegram["fivegram"].apply(eval)
df_threegram["sixgram"] = df_threegram["sixgram"].apply(eval)
df_threegram["sevengram"] = df_threegram["sevengram"].apply(eval)
df_threegram["eightgram"] = df_threegram["eightgram"].apply(eval)
df_threegram["ninegram"] = df_threegram["ninegram"].apply(eval)
df_threegram["tengram"] = df_threegram["tengram"].apply(eval)

In [26]:
threegram = df_threegram["threegram"].to_list()
frequency_list = df_threegram["frequency"].to_list()
fourgram_list = df_threegram["fourgram"].to_list()
fivegram_list = df_threegram["fivegram"].to_list()
sixgram_list = df_threegram["sixgram"].to_list()
sevengram_list = df_threegram["sevengram"].to_list()
eightgram_list = df_threegram["eightgram"].to_list()
ninegram_list = df_threegram["ninegram"].to_list()
tengram_list = df_threegram["tengram"].to_list()

In [ ]:
firestore_threegram_data = zip(threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
threegram_dict = {}
for threegram,frequency_list,fourgram_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_threegram_data:
    dict_var = {f"{threegram}":{"photo":"","frequency":frequency_list,"fourgram":fourgram_list,"fivegram":fivegram_list,"sixgram":
    sixgram_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    threegram_dict.update(dict_var)    

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("threegram_data").document(f"details").set(threegram_dict)

##### Fourgram

In [ ]:
df_fourgram

In [27]:
df_fourgram["fivegram"] = df_fourgram["fivegram"].apply(eval)
df_fourgram["sixgram"] = df_fourgram["sixgram"].apply(eval)
df_fourgram["sevengram"] = df_fourgram["sevengram"].apply(eval)
df_fourgram["eightgram"] = df_fourgram["eightgram"].apply(eval)
df_fourgram["ninegram"] = df_fourgram["ninegram"].apply(eval)
df_fourgram["tengram"] = df_fourgram["tengram"].apply(eval)

In [28]:
fourgram = df_fourgram["fourgram"].to_list()
frequency_list = df_fourgram["frequency"].to_list()
fivegram_list = df_fourgram["fivegram"].to_list()
sixgram_list = df_fourgram["sixgram"].to_list()
sevengram_list = df_fourgram["sevengram"].to_list()
eightgram_list = df_fourgram["eightgram"].to_list()
ninegram_list = df_fourgram["ninegram"].to_list()
tengram_list = df_fourgram["tengram"].to_list()

In [ ]:
firestore_fourgram_data = zip(fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
fourgram_dict = {}
for fourgram,frequency_list,fivegram_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fourgram_data:
    dict_var = {f"{fourgram}":{"photo":"","frequency":frequency_list,"fivegram":fivegram_list,"sixgram":sixgram_list,
    "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    fourgram_dict.update(dict_var)    

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fourgram_data").document(f"details").set(fourgram_dict)

##### Fivegram

In [ ]:
df_fivegram

In [29]:
df_fivegram["sixgram"] = df_fivegram["sixgram"].apply(eval)
df_fivegram["sevengram"] = df_fivegram["sevengram"].apply(eval)
df_fivegram["eightgram"] = df_fivegram["eightgram"].apply(eval)
df_fivegram["ninegram"] = df_fivegram["ninegram"].apply(eval)
df_fivegram["tengram"] = df_fivegram["tengram"].apply(eval)

In [30]:
fivegram = df_fivegram["fivegram"].to_list()
frequency_list = df_fivegram["frequency"].to_list()
sixgram_list = df_fivegram["sixgram"].to_list()
sevengram_list = df_fivegram["sevengram"].to_list()
eightgram_list = df_fivegram["eightgram"].to_list()
ninegram_list = df_fivegram["ninegram"].to_list()
tengram_list = df_fivegram["tengram"].to_list()

In [ ]:
firestore_fivegram_data = zip(fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
fivegram_dict = {}
for fivegram,frequency_list,sixgram_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_fivegram_data:
    dict_var = {f"{fivegram}":{"photo":"","frequency":frequency_list,"sixgram":sixgram_list,
    "sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    fivegram_dict.update(dict_var)   

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("fivegram_data").document(f"details").set(fivegram_dict)

##### Sixgram

In [ ]:
df_sixgram

In [31]:
df_sixgram["sevengram"] = df_sixgram["sevengram"].apply(eval)
df_sixgram["eightgram"] = df_sixgram["eightgram"].apply(eval)
df_sixgram["ninegram"] = df_sixgram["ninegram"].apply(eval)
df_sixgram["tengram"] = df_sixgram["tengram"].apply(eval)

In [32]:
sixgram = df_sixgram["sixgram"].to_list()
frequency_list = df_sixgram["frequency"].to_list()
sevengram_list = df_sixgram["sevengram"].to_list()
eightgram_list = df_sixgram["eightgram"].to_list()
ninegram_list = df_sixgram["ninegram"].to_list()
tengram_list = df_sixgram["tengram"].to_list()

In [ ]:
firestore_sixgram_data = zip(sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
sixgram_dict = {}
for sixgram,frequency_list,sevengram_list,eightgram_list,ninegram_list,tengram_list in firestore_sixgram_data:
    dict_var = {f"{sixgram}":{"photo":"","frequency":frequency_list,"sevengram":sevengram_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    sixgram_dict.update(dict_var)   

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sixgram_data").document(f"details").set(sixgram_dict)

##### Sevengram

In [ ]:
df_sevengram

In [33]:
df_sevengram["eightgram"] = df_sevengram["eightgram"].apply(eval)
df_sevengram["ninegram"] = df_sevengram["ninegram"].apply(eval)
df_sevengram["tengram"] = df_sevengram["tengram"].apply(eval)

In [34]:
sevengram = df_sevengram["sevengram"].to_list()
frequency_list = df_sevengram["frequency"].to_list()
eightgram_list = df_sevengram["eightgram"].to_list()
ninegram_list = df_sevengram["ninegram"].to_list()
tengram_list = df_sevengram["tengram"].to_list()

In [ ]:
firestore_sevengram_data = zip(sevengram,frequency_list,eightgram_list,ninegram_list,tengram_list)

In [ ]:
sevengram_dict = {}
for sevengram,frequency_list,eightgram_list,ninegram_list,tengram_list in firestore_sevengram_data:
    dict_var = {f"{sevengram}":{"photo":"","frequency":frequency_list,"eightgram":eightgram_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    sevengram_dict.update(dict_var)  

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("sevengram_data").document(f"details").set(sevengram_dict)

##### Eightgram

In [ ]:
df_eightgram

In [35]:
df_eightgram["ninegram"] = df_eightgram["ninegram"].apply(eval)
df_eightgram["tengram"] = df_eightgram["tengram"].apply(eval)

In [36]:
eightgram = df_eightgram["eightgram"].to_list()
frequency_list = df_eightgram["frequency"].to_list()
ninegram_list = df_eightgram["ninegram"].to_list()
tengram_list = df_eightgram["tengram"].to_list()

In [ ]:
firestore_eightgram_data = zip(eightgram,frequency_list,ninegram_list,tengram_list)

In [ ]:
eightgram_dict = {}
for eightgram,frequency_list,ninegram_list,tengram_list in firestore_eightgram_data:
    dict_var = {f"{eightgram}":{"photo":"","frequency":frequency_list,"ninegram":ninegram_list,"tengram":tengram_list,"other":[]}}
    eightgram_dict.update(dict_var)  

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("eightgram_data").document(f"details").set(eightgram_dict)

##### Ninegram

In [ ]:
df_ninegram

In [37]:
df_ninegram["tengram"] = df_ninegram["tengram"].apply(eval)

In [38]:
ninegram = df_ninegram["ninegram"].to_list()
frequency_list = df_ninegram["frequency"].to_list()
tengram_list = df_ninegram["tengram"].to_list()

In [ ]:
firestore_ninegram_data = zip(ninegram,frequency_list,tengram_list)

In [ ]:
ninegram_dict = {}
for ninegram,frequency_list,tengram_list in firestore_ninegram_data:
    dict_var = {f"{ninegram}":{"photo":"","frequency":frequency_list,"tengram":tengram_list,"other":[]}}
    ninegram_dict.update(dict_var)  

In [ ]:
fire_db.collection("text_selection").document(f"{lang_folder.capitalize()}").collection("ninegram_data").document(f"details").set(ninegram_dict)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass